# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]


data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [5]:
recommender = MainRecommender(data_train_lvl_1)

In [6]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 1044078,
 1106523,
 883932,
 871756,
 5569230,
 1025535,
 8090537,
 1000753,
 1022428,
 832678,
 844179,
 1046545,
 850102,
 1081177,
 841220,
 823990,
 12262978,
 947858,
 1051323,
 8090521,
 925862,
 6534480,
 854852,
 1116376,
 9835223,
 965267,
 896613,
 865528,
 999714,
 895166,
 1068292,
 870547,
 905087,
 823704,
 999779,
 865456,
 863447,
 879504,
 1070702,
 1107420,
 910032,
 850925,
 835300,
 949965,
 1090931,
 1119454,
 1104349,
 828106,
 1029743,
 9836106,
 831517,
 956666,
 1072685,
 986912,
 1037863,
 896862,
 874972,
 940090,
 5569845,
 1132771,
 12301839,
 847790,
 963727,
 937292,
 5585510,
 1133312,
 1009770,
 1097458,
 835530,
 1131344,
 999270,
 1094833,
 1016800,
 995896,
 1001702,
 826666,
 8065410,
 861279,
 866211,
 857006,
 5568995,
 835098,
 12731517,
 1134678,
 923149,
 1038663,
 1119784,
 851188,
 917816,
 938138,
 825343,
 891423,
 902172,
 965766,
 948650,
 993638,
 926523,
 1004390,
 973181,
 1042907,
 8090509,
 1003616,
 1128244,
 960318,
 9521

In [7]:
recommender.get_own_recommendations(2375, N=200)

[1029743,
 1106523,
 5569230,
 916122,
 844179,
 1044078,
 1126899,
 1070820,
 1127831,
 866211,
 8090521,
 878996,
 8090537,
 5569471,
 1004906,
 854852,
 899624,
 986912,
 933835,
 1075368,
 1081177,
 6034857,
 5585510,
 965267,
 834117,
 940947,
 983584,
 12810393,
 913210,
 874972,
 5569845,
 5568378,
 999858,
 908318,
 985999,
 901062,
 1040807,
 1018740,
 951412,
 1101010,
 1105488,
 1037840,
 880150,
 1043751,
 857503,
 1122358,
 1132771,
 823704,
 854405,
 909714,
 993638,
 930118,
 839419,
 971922,
 910032,
 863447,
 835098,
 865456,
 976199,
 12301109,
 1070702,
 5569374,
 8090532,
 872137,
 883003,
 1023720,
 1012587,
 1000753,
 893018,
 1024306,
 897954,
 1042438,
 907631,
 5568729,
 1068719,
 1137775,
 1087102,
 944534,
 1020581,
 999270,
 832678,
 838186,
 1138443,
 999104,
 1112238,
 1050851,
 902172,
 965766,
 12301100,
 952163,
 957951,
 8090509,
 852856,
 885863,
 1056509,
 845307,
 1062002,
 944836,
 819978,
 957736,
 999779,
 1027168,
 1051323,
 8065410,
 1026118,
 

In [8]:
recommender.get_similar_items_recommendation(880150, N=200)

[1029743,
 1106523,
 5569230,
 916122,
 844179,
 1044078,
 1126899,
 1070820,
 1127831,
 866211,
 8090521,
 878996,
 8090537,
 5569471,
 1004906,
 854852,
 899624,
 986912,
 933835,
 1075368,
 1081177,
 6034857,
 5585510,
 965267,
 834117,
 940947,
 983584,
 12810393,
 913210,
 874972,
 5569845,
 5568378,
 999858,
 908318,
 985999,
 901062,
 1040807,
 1018740,
 951412,
 1101010,
 1105488,
 1037840,
 880150,
 1043751,
 857503,
 1122358,
 1132771,
 823704,
 854405,
 909714,
 993638,
 930118,
 839419,
 971922,
 910032,
 863447,
 835098,
 865456,
 976199,
 12301109,
 1070702,
 5569374,
 8090532,
 872137,
 883003,
 1023720,
 1012587,
 1000753,
 893018,
 1024306,
 897954,
 1042438,
 907631,
 5568729,
 1068719,
 1137775,
 1087102,
 944534,
 1020581,
 999270,
 832678,
 838186,
 1138443,
 999104,
 1112238,
 1050851,
 902172,
 965766,
 12301100,
 952163,
 957951,
 8090509,
 852856,
 885863,
 1056509,
 845307,
 1062002,
 944836,
 819978,
 957736,
 999779,
 1027168,
 1051323,
 8065410,
 1026118,
 

In [9]:
recommender.get_similar_users_recommendation(2375, N=200)

[2381,
 237,
 1016,
 170,
 1614,
 1217,
 2256,
 897,
 1897,
 942,
 1494,
 2218,
 205,
 857,
 1733,
 1936,
 551,
 196,
 192,
 1240,
 1602,
 651,
 2400,
 1079,
 491,
 1664,
 2052,
 177,
 2224,
 1546,
 644,
 241,
 980,
 356,
 1871,
 1182,
 1314,
 669,
 348,
 1195,
 596,
 154,
 2362,
 1309,
 126,
 556,
 813,
 2285,
 191,
 103,
 249,
 1700,
 7,
 16,
 1770,
 792,
 1301,
 849,
 1926,
 635,
 699,
 317,
 1929,
 1760,
 2141,
 1442,
 211,
 1956,
 1981,
 1091,
 2223,
 479,
 934,
 2490,
 985,
 1374,
 1343,
 2411,
 2493,
 453,
 254,
 1513,
 398,
 2335,
 344,
 2347,
 488,
 2462,
 1263,
 1801,
 974,
 2215,
 1688,
 1840,
 58,
 1596,
 930,
 1780,
 852,
 1366,
 1775,
 352,
 1446,
 372,
 855,
 1932,
 533,
 502,
 1347,
 1386,
 1751,
 763,
 2252,
 1953,
 862,
 111,
 2424,
 1329,
 462,
 1570,
 1990,
 1804,
 1977,
 1410,
 268,
 1361,
 2003,
 296,
 613,
 1909,
 2199,
 2193,
 349,
 2264,
 1074,
 184,
 1168,
 1772,
 677,
 1744,
 2255,
 2167,
 1900,
 1223,
 1492,
 2491,
 1560,
 406,
 856,
 2084,
 2104,
 218,
 883

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [10]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.count()

user_id    2154
actual     2154
dtype: int64

In [12]:
recommend_list= []
mean_recall = []
for user, bought_list in zip(result_lvl_1['user_id'],result_lvl_1['actual']):
    recommend_list = recommender.get_similar_items_recommendation(user, N=200)
    # print(f'Recall {user} равен {recall_at_k(recommend_list, bought_list, k=200)}')
    mean_recall.append(recall_at_k(recommend_list, bought_list, k=200))

print(f'Средний Recall равен {np.array(mean_recall).mean() * 100}')

Средний Recall равен 8.538228845929211


In [30]:
recommend_list= []
mean_precision_at_k = []
for user, bought_list in zip(result_lvl_1['user_id'],result_lvl_1['actual']):
    recommend_list = recommender.get_similar_items_recommendation(user, N=200)
    mean_precision_at_k.append(precision_at_k(recommend_list, bought_list, k=200))
    
print(f'Средний precision_at_k равен {np.array(mean_precision_at_k).mean() * 100}')

Средний precision_at_k равен 2.3999535747446608


### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [13]:
n_items_before = data_train_lvl_2['item_id'].nunique()

data_train_lvl_2 = prefilter_items(data_train_lvl_2, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_2['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 27649 to 5001


In [14]:
data_train_lvl_2 = data_train_lvl_2.astype({'user_id': 'int32','basket_id': 'int64','day': 'int32','item_id': 'int32','quantity': 'int32','user_id': 'int32',})

In [15]:
# Создаем фичу число покупок юзером за месяц
def create_pur_per_month(data):
    values = {}
    df = data.drop_duplicates(['basket_id'])
    for user in data['user_id']:
        values[user] = df.loc[df['user_id'] == user]['basket_id'].count() / 4
        
    for user in values.keys():
        data.loc[data_train_lvl_2['user_id'] == user, 'pur_per_month'] = values[user]
    return data

data_train_lvl_2 = create_pur_per_month(data_train_lvl_2)

In [16]:
# Создаем фичу число покупок товара за месяц
def create_sel_per_month(data):
    values = {}
    df = data
    for item in data['item_id']:
        values[item] = df.loc[df['item_id'] == item]['basket_id'].count() * df.loc[df['item_id'] == item]['quantity'].sum() / 4
        
    for item in values.keys():
        data.loc[data_train_lvl_2['item_id'] == item, 'sell_per_month'] = values[item]
    return data

data_train_lvl_2 = create_sel_per_month(data_train_lvl_2)

In [17]:
data_train_lvl_2

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,pur_per_month,sell_per_month
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0,2.50,1.00,600.00
2107473,2021,40618753059,594,1019142,2,5.00,443,-1.98,101,86,0.0,0.0,2.50,1.00,30.00
2107476,2021,40618753059,594,9835223,1,9.27,443,-3.63,101,86,0.0,0.0,9.27,1.00,304.00
2108010,1753,40618809138,594,999999,1,29.99,345,0.00,8,86,0.0,0.0,29.99,5.00,91870339.00
2108685,2120,40629515323,594,999999,1,2.09,136,0.00,830,86,0.0,0.0,2.09,2.50,91870339.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282311,2088,41297771158,635,13877226,1,15.65,304,-5.25,1258,91,0.0,0.0,15.65,3.25,8.75
2282315,1541,41297771180,635,999999,1,5.19,304,0.00,1301,91,0.0,0.0,5.19,2.00,91870339.00
2282316,1168,41297772063,635,836262,1,12.40,304,0.00,1526,91,0.0,0.0,12.40,1.25,4.00
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0,3.00,1.50,49.00


Значения в 91870339 для не популярных товаров

In [23]:
def create_xsandys(data_train_lvl_2):
    users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
    users_lvl_2.columns = ['user_id']
    
    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1

    # Пока только warm start
    train_users = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

    users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
    s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'

    users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
    users_lvl_2['drop'] = 1  # фиктивная пересенная
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    
    X_train = targets_lvl_2.drop('target', axis=1)
    y_train = targets_lvl_2[['target']]
    
    cat_feats = X_train.columns[2:].tolist()
    X_train[cat_feats] = X_train[cat_feats].astype('category')
    
    return X_train, y_train, cat_feats
    

In [24]:
X_train, y_train, cat_feats = create_xsandys(data_train_lvl_2)

In [25]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

C:\Program Files\Python37\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python37\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Wall time: 459 ms


In [26]:
X_val, y_val, cat_feats = create_xsandys(data_val_lvl_2)

In [33]:
print(f'Средний precision_at_k равен {precision_at_k(train_preds,y_val, k=200) / 100}')

Средний precision_at_k равен 5.9157


In [ ]:
При использовании 2ух-уровневой модели 

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)